## Imports and Data

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

england_matches = pd.read_json('new_data/matches/matches_England.json')
teams = pd.read_json('new_data/teams.json')
england_events = pd.read_json('new_data/events/events_England.json')
players = pd.read_json('new_data/players.json')

## Team Analysis
### Team class for accessing team attributes (id, matches, players, events etc.) 


In [3]:
class team:
    def __init__(self, name:str):
        self.name = name
        self.team_id = self.get_team_id()
        self.matches = self.get_matches()
        self.players = self.get_players()
        self.all_events = self.get_all_events()
    
    def get_team_id(self):
        return int(teams[teams['name'] == self.name]['wyId'])
        
    def get_matches(self, opposition_team=None):
        matches = pd.DataFrame(columns=england_matches.columns)
        specific_matches = pd.DataFrame(columns=england_matches.columns)
        
        for index,row in england_matches.iterrows():
            teams_in_match = [int(team) for team in list(row['teamsData'].keys())]
            if opposition_team is None:
                if self.team_id in teams_in_match:
                    matches = matches.append(row)
            else:
                if self.team_id in teams_in_match and opposition_team.team_id in teams_in_match:
                    specific_matches = specific_matches.append(row)
                
        if opposition_team is None:     
            return matches
        else:
            return specific_matches
    
    def get_players(self):
        return players[players['currentTeamId'] == self.team_id]
    
    def get_all_events(self):
        return england_events[england_events['teamId'] == self.team_id]

## Match Analysis

In [11]:
from matplotlib.patches import Arc



def draw_pitch():
    fig = plt.figure()
    fig.set_size_inches(14,7)

    ax = plt.subplot(111)
    
    #touchline
    plt.plot([0,0], [-5,105], 'k') #left length
    plt.plot([0,100], [105,105], 'k') #upper width
    plt.plot([100,100], [-5,105], 'k') #right length
    plt.plot([100,0], [-5,-5], 'k') #lower width
    plt.plot([50,50], [-5,105], 'k') #halfway line

    #center
    center_circle = plt.Circle((50,50), radius=7, color='black', fill=False)
    center_spot = plt.Circle((50,50), radius=0.7, color='black', fill=True)
    ax.add_artist(center_circle)
    ax.add_artist(center_spot)

    #left 18 yard box
    plt.plot([0,12], [32,32], 'k')
    plt.plot([0,12], [69,69], 'k')
    plt.plot([12,12], [32,69], 'k')
    left_d = Arc((6, 50), width=18.3, height=20, theta1=310, theta2=50, color='black')
    ax.add_patch(left_d)

    #left 6 yard box
    plt.plot([0,4], [41.25,41.25], 'k')
    plt.plot([0,4], [59.75,59.75], 'k')
    plt.plot([4,4], [59.75,41.25], 'k')

    #right 18 yard box
    plt.plot([100,88], [32,32], 'k')
    plt.plot([100,88], [69,69], 'k')
    plt.plot([88,88], [32,69], 'k')
    right_d = Arc((94, 50), width=18.3, height=20, theta1=130, theta2=230, color='black')
    ax.add_patch(right_d)

    #right 6 yard box
    plt.plot([100,96], [41.25,41.25], 'k')
    plt.plot([100,96], [59.75,59.75], 'k')
    plt.plot([96,96], [59.75,41.25], 'k')

#dataframe should be an 'events' dataframe
def draw_heatmap(dataframe):
    sns.kdeplot(dataframe['x_start'], dataframe['y_start'], shade=True, shade_lowest=False, cmap='Greens', n_levels = 30)
    plt.xlim, plt.ylim = (0,100), (0,100)

    plt.axis('off')
    plt.show()

In [51]:
from ipywidgets import interact, interactive, interact_manual, widgets
import warnings 
warnings.filterwarnings('ignore')

team_selector_options = ['Arsenal', 'Leicester City']

team_1, team_2 = None, None
matches = None
target_match = None
target_team = None

team_1_selector = widgets.Dropdown(options=team_selector_options, description='Team 1:')
team_2_selector = widgets.Dropdown(options=team_selector_options, description='Team 2:')

def select_teams(team_1_selector, team_2_selector):
    global team_1, team_2, matches
    
    team_1 = team(team_1_selector)
    team_2 = team(team_2_selector)
    matches = team_1.get_matches(team_2)
    
    match_selector = interact(select_match, match_selector=matches['label'].tolist(), description='Match')
    
def select_match(match_selector) -> pd.DataFrame:
    global matches, target_match
    
    target_match = matches[matches['label'] == match_selector]
#     display(target_match)
#     target_team_selector = interact_manual(select_team, team_selector=team_selector_options)
    target_team_selector = interact(select_team, team_selector=team_selector_options)

def select_team(team_selector):
    global team_1, team_2, target_match, target_team
    
    target_team = team_1 if team_selector == team_1.name else team_2
    range_selector = widgets.SelectionRangeSlider(options=[0,45,90], index=(0, 2), description='Interval', disabled=False)
    interact_manual(select_range, range_selector=range_selector)
    
def select_range(range_selector):
    global target_team, target_match
    
    events = target_team.all_events[target_team.all_events['matchId'] == int(target_match['wyId'])]
    events = events[events['eventSec'] >= range_selector[0] * 60]
    events = events[events['eventSec'] <= range_selector[1] * 60]
    passes = events[events['eventName'] == 'Pass']
    passes['x_start'] = passes.apply(lambda row: row['positions'][0]['x'], axis=1)
    passes['y_start'] = passes.apply(lambda row: row['positions'][0]['y'], axis=1)
    
    draw_pitch()
    draw_heatmap(passes)
    
    

    
interact_manual(select_teams, team_1_selector=team_1_selector, team_2_selector=team_2_selector)

interactive(children=(Dropdown(description='Team 1:', options=('Arsenal', 'Leicester City'), value='Arsenal'),…

<function __main__.select_teams(team_1_selector, team_2_selector)>

In [38]:
arsenal = team('Arsenal')
# arsenal.players
# arsenal.players[arsenal.players['wyId'] == 370224]
arsenal.all_events[arsenal.all_events['eventSec'] <= 45*60]

,eventId,eventName,eventSec,id,matchId,matchPeriod,playerId,positions,subEventId,subEventName,tags,teamId
0,8,Pass,2.758649,177959171,2499719,1H,25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",85,Simple pass,[{'id': 1801}],1609
1,8,Pass,4.946850,177959172,2499719,1H,370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",83,High pass,[{'id': 1801}],1609
2,8,Pass,6.542188,177959173,2499719,1H,3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",82,Head pass,[{'id': 1801}],1609
3,8,Pass,8.143395,177959174,2499719,1H,120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",82,Head pass,[{'id': 1801}],1609
4,8,Pass,10.302366,177959175,2499719,1H,167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",85,Simple pass,[{'id': 1801}],1609
5,8,Pass,12.548934,177959177,2499719,1H,3319,"[{'y': 88, 'x': 72}, {'y': 75, 'x': 77}]",85,Simple pass,[{'id': 1802}],1609
8,1,Duel,14.765321,177961218,2499719,1H,0,"[{'y': 85, 'x': 61}, {'y': 80, 'x': 67}]",10,Air duel,"[{'id': 703}, {'id': 1801}]",1609
9,8,Pass,15.320341,177959178,2499719,1H,167145,"[{'y': 80, 'x': 67}, {'y': 61, 'x': 59}]",82,Head pass,"[{'id': 1401}, {'id': 1801}]",1609
10,8,Pass,18.051875,177959179,2499719,1H,49876,"[{'y': 61, 'x': 59}, {'y': 45, 'x': 45}]",82,Head pass,[{'id': 1801}],1609
11,8,Pass,20.426527,177959180,2499719,1H,14869,"[{'y': 45, 'x': 45}, {'y': 54, 'x': 71}]",83,High pass,[{'id': 1801}],1609
